In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
189,ABW,North America,Aruba,2020-09-23,3665.0,78.0,73.286,25.0,1.0,0.429,...,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29,NaN
457,AFG,Asia,Afghanistan,2020-09-23,39145.0,71.0,41.429,1446.0,2.0,1.429,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
643,AGO,Africa,Angola,2020-09-23,4236.0,119.0,113.857,155.0,1.0,3.000,...,5819.495,NaN,276.045,3.94,NaN,NaN,26.664,NaN,61.15,0.581


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-09-22,France,468698.0,0.0,31365.0,0.0,0.0,0.0
0,2020-09-22,China,90406.0,0.0,4742.0,0.0,0.0,0.0
0,2020-09-22,Italy,300918.0,0.0,35755.0,0.0,0.0,0.0
0,2020-09-22,Spain,680180.0,0.0,30686.0,0.0,0.0,0.0
0,2020-09-22,United States,6927494.0,0.0,200867.0,0.0,0.0,0.0
0,2020-09-22,World,31925403.0,0.0,975336.0,0.0,0.0,0.0
0,2020-09-22,United Kingdom,401562.0,0.0,41834.0,0.0,0.0,0.0
0,2020-09-22,Germany,275044.0,0.0,9401.0,0.0,0.0,0.0
0,2020-09-22,Iran,428420.0,0.0,24687.0,0.0,0.0,0.0
0,2020-09-22,Turkey,305796.0,0.0,7623.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-09-23,France,468698.0,468069.0,31365.0,31416.0,629.0,-51.0
0,2020-09-23,China,90406.0,90399.0,4742.0,4737.0,7.0,5.0
0,2020-09-23,Italy,300918.0,300897.0,35755.0,35738.0,21.0,17.0
0,2020-09-22,Spain,680180.0,682267.0,30686.0,30904.0,-2087.0,-218.0
0,2020-09-23,United States,6927494.0,6896274.0,200867.0,200807.0,31220.0,60.0
0,2020-09-23,World,31925403.0,31658573.0,975336.0,971869.0,266830.0,3467.0
0,2020-09-23,United Kingdom,401562.0,403551.0,41834.0,41825.0,-1989.0,9.0
0,2020-09-23,Germany,275044.0,275927.0,9401.0,9409.0,-883.0,-8.0
0,2020-09-23,Iran,428420.0,429193.0,24687.0,24656.0,-773.0,31.0
0,2020-09-23,Turkey,305796.0,306302.0,7623.0,7639.0,-506.0,-16.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")